In [1]:
import valurap.commands as vac
import valurap.printer as vp
import valurap.axes as vax
import valurap.apgs as vap
import valurap.asg as vas
import valurap.spi as vs

import valurap.path_planning as vapp

from importlib import reload
reload(vs)
reload(vac)
reload(vas)
reload(vax)
reload(vap)
reload(vp)
reload(vapp)

prn = vp.Valurap()
prn.setup()
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22

/home/seva/.virtualenvs/valurap3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/seva/.virtualenvs/valurap3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Executing reset code
Update axes
Reset OLED
Setup done


In [2]:
prn.home()

SetPositions: {'Z': 0}
Move: {'Z': 30000}
homing: [<valurap.axes.AxeX1 object at 0xab561a50>, <valurap.axes.AxeX2 object at 0xab578c50>, <valurap.axes.AxeY object at 0xab578cb0>]
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X1': 0, 'X2': 0, 'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab561a50>, <valurap.axes.AxeX2 object at 0xab578c50>, <valurap.axes.AxeY object at 0xab578cb0>]
X1 already on endstop
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X2': 0, 'Y': 0}
homing: [<valurap.axes.AxeX1 object at 0xab561a50>, <valurap.axes.AxeX2 object at 0xab578c50>, <valurap.axes.AxeY object at 0xab578cb0>]
X1 already on endstop
Warning, explicit X in segment
Y already on endstop
SetPositions: {'X2': 0}
homing: [<valurap.axes.AxeX1 object at 0xab561a50>, <valurap.axes.AxeX2 object at 0xab578c50>, <valurap.axes.AxeY object at 0xab578cb0>]
X1 already on endstop
X2 already on endstop
Y already on endst

In [51]:
prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.update_axes_config()

prn.moveto(X1=0)
prn.moveto(Y=0)

Save position: X1 12698.84765625
Save position: X2 -13501.1484375
Save position: Y -20001.1484375
SetPositions: {'X1': 12698.84765625, 'X2': -13501.1484375, 'Y': -20001.1484375}
MoveTo: {'X1': 0}
Save position: X1 12697.84765625
Save position: X2 -13502.1484375
Save position: Y -20002.1484375
Move: {'X1': -12697.84765625}
MoveTo: {'Y': 0}
Save position: X1 -0.4140625
Move: {'Y': 20002.1484375}


In [ ]:
#prn.moveto(X1=0)
#prn.moveto(Y=0)

#prn.move(Z=-1600*30)
x1, y1 = [255.1652, 288.87604]
x2, y2 = [339.1803, 288.58218]

x3, y3 = [339.1803, 288.58218]
x4, y4 = [339.62244, 372.909]

x, y = [358.5733, 240.30933]


dx = (x - 320)/(x2 - x1)*10
dy = (y - 240)/(y4 - y3)*10
display(dx, dy)
z1 = [
    [320.05545, 240.03896],
    [320.58078, 243.7491],
    [321.13934, 247.78864],
    [321.85776, 252.33138]
]

dx = -40
dy = 0
prn.move(X2=-80*dx, Y=-80*dy)
#prn.move(Z=-1600*30)
#prn.move(X2=-80*10)


In [ ]:
prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.axe_e2.enabled = False
prn.update_axes_config()

prn.move(E1=-100000)
#prn.move(E1=-20000)
prn.moveto(X1=1000)
#prn.moveto(X2=0)
prn.moveto(Y=-3000)
prn.moveto(Z=5040)
prn.moveto(Z=5040-1600*1.5)

In [ ]:
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22
display(22/3)
display(3*3/1.75/1.75)

In [ ]:
display(prn.axe_x1.last_pos)
display(prn.axe_x2.last_pos)
display(prn.axe_y.last_pos)

In [7]:
# open-loop
import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs=apgs)

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

prn.axe_z.enabled = True
prn.axe_y.enabled = True
prn.axe_x1.enabled = True
prn.axe_e1.enabled = True
prn.update_axes_config()

start = None
apg_map = None

for layer in range(1, 4):
    with open("test18_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    if len(p) == 1 and p[0][0] == "do_home":
        break
        
    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    if start is None:
        start = p[0]

    pr_opt = [
        [1, [
            vas.ProfileSegment(apgs["X"], x=p[0][1]["X"] * spm),
            vas.ProfileSegment(apgs["Y"], x=p[0][1]["Y"] * spm),
            vas.ProfileSegment(apgs["Z"], x=0),
        ]]
    ]
    cms, meta, segments = p[1]
    if apg_map is None:
        apg_map = meta["map"]
        
    codes.append(prn.asg.gen_map_code(meta["map"]))
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    print(len(path_code))
    codes.append(path_code)
    codes.append(prn.asg.gen_map_code({"Z": "Z"}))
    codes.append(up_code)

prn.update_axes_config()
fullcode = []
for code in codes:
    fullcode.extend(code[:-1])
fullcode.extend(code[-1:])

print(len(fullcode))

prn.setup()

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.axe_e2.enabled = False
prn.update_axes_config()

prn.move(E1=-100000)
#prn.move(E1=-20000)
prn.moveto(X1=0)
prn.moveto(Y=0)
#prn.moveto(X2=0)
prn.moveto(Y=start[1]["Y"] * 80, X1=start[1]["X"] * 80)
prn.moveto(Z=5040)
prn.moveto(Z=5040-1600*1.5)

for k, v in apg_map.items():
    prn.axes[k].apg = prn.apgs[v]
    
#print(start[2])
#prn.moveto(X1=start[1]["X"] * 80, Y=start[1]["Y"] * 80)

try:
    prn.exec_long_code(fullcode, splits=2000, verbose=True)
    
    for axe in prn.axes.values():
        axe.apg = None
        axe.enabled = False
    
    prn.update_axes_config()
    prn.home()
    prn.setup()
except:
    prn.setup()
    


Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
257829
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
253499
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled
Axe Z mapped to Z
Axe Z enabled
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
Map: {'X1': 'X', 'Y': 'Y', 'E1': 'Z'}
Axe X1 mapped to X
Axe X1 enabled
Axe Y mapped to Y
Axe Y enabled
Axe E1 mapped to Z
Axe E1 enabled
Axe Z enabled
248588
Map: {'Z': 'Z'}
Axe X1 enabled
Axe Y enabled
Axe E1 enabled


too small free_buf: 1546
Busy: 1 Wait: 1 Error: 2 PC: 3415 Cur addr: 1808 Cur len: 750019
too small free_buf: 1607
Busy: 1 Wait: 1 Error: 2 PC: 3614 Cur addr: 1808 Cur len: 750019
too small free_buf: 1806
Busy: 1 Wait: 1 Error: 2 PC: 3684 Cur addr: 1808 Cur len: 750019
too small free_buf: 1876
Busy: 1 Wait: 1 Error: 2 PC: 3746 Cur addr: 1808 Cur len: 750019
too small free_buf: 1938
Busy: 1 Wait: 1 Error: 2 PC: 4095 Cur addr: 1808 Cur len: 750019
free_buf: 2287
sent 2000 commands to addr 1808, left 748019
Busy: 1 Wait: 1 Error: 2 PC: 4491 Cur addr: 3808 Cur len: 748019
too small free_buf: 683
Busy: 1 Wait: 1 Error: 2 PC: 4515 Cur addr: 3808 Cur len: 748019
too small free_buf: 707
Busy: 1 Wait: 1 Error: 2 PC: 4531 Cur addr: 3808 Cur len: 748019
too small free_buf: 723
Busy: 1 Wait: 1 Error: 2 PC: 4554 Cur addr: 3808 Cur len: 748019
too small free_buf: 746
Busy: 1 Wait: 1 Error: 2 PC: 4570 Cur addr: 3808 Cur len: 748019
too small free_buf: 762
Busy: 1 Wait: 1 Error: 2 PC: 4585 Cur addr: 3

too small free_buf: 886
Busy: 1 Wait: 1 Error: 2 PC: 14753 Cur addr: 5616 Cur len: 738019
too small free_buf: 945
Busy: 1 Wait: 1 Error: 2 PC: 14772 Cur addr: 5616 Cur len: 738019
too small free_buf: 964
Busy: 1 Wait: 1 Error: 2 PC: 14962 Cur addr: 5616 Cur len: 738019
too small free_buf: 1154
Busy: 1 Wait: 1 Error: 2 PC: 14979 Cur addr: 5616 Cur len: 738019
too small free_buf: 1171
Busy: 1 Wait: 1 Error: 2 PC: 15002 Cur addr: 5616 Cur len: 738019
too small free_buf: 1194
Busy: 1 Wait: 1 Error: 2 PC: 15020 Cur addr: 5616 Cur len: 738019
too small free_buf: 1212
Busy: 1 Wait: 1 Error: 2 PC: 15046 Cur addr: 5616 Cur len: 738019
too small free_buf: 1238
Busy: 1 Wait: 1 Error: 2 PC: 15214 Cur addr: 5616 Cur len: 738019
too small free_buf: 1406
Busy: 1 Wait: 1 Error: 2 PC: 15262 Cur addr: 5616 Cur len: 738019
too small free_buf: 1454
Busy: 1 Wait: 1 Error: 2 PC: 15279 Cur addr: 5616 Cur len: 738019
too small free_buf: 1471
Busy: 1 Wait: 1 Error: 2 PC: 15476 Cur addr: 5616 Cur len: 738019
to

too small free_buf: 1580
Busy: 1 Wait: 1 Error: 2 PC: 23452 Cur addr: 5424 Cur len: 730019
too small free_buf: 1644
Busy: 1 Wait: 1 Error: 2 PC: 23596 Cur addr: 5424 Cur len: 730019
too small free_buf: 1788
Busy: 1 Wait: 1 Error: 2 PC: 23612 Cur addr: 5424 Cur len: 730019
too small free_buf: 1804
Busy: 1 Wait: 1 Error: 2 PC: 23815 Cur addr: 5424 Cur len: 730019
too small free_buf: 2007
Busy: 1 Wait: 1 Error: 2 PC: 23833 Cur addr: 5424 Cur len: 730019
free_buf: 2025
sent 2000 commands to addr 5424, left 728019
Busy: 1 Wait: 1 Error: 2 PC: 24212 Cur addr: 7424 Cur len: 728019
too small free_buf: 404
Busy: 1 Wait: 1 Error: 2 PC: 24228 Cur addr: 7424 Cur len: 728019
too small free_buf: 420
Busy: 1 Wait: 1 Error: 2 PC: 24251 Cur addr: 7424 Cur len: 728019
too small free_buf: 443
Busy: 1 Wait: 1 Error: 2 PC: 24267 Cur addr: 7424 Cur len: 728019
too small free_buf: 459
Busy: 1 Wait: 1 Error: 2 PC: 24290 Cur addr: 7424 Cur len: 728019
too small free_buf: 482
Busy: 1 Wait: 1 Error: 2 PC: 24306 

too small free_buf: 781
Busy: 1 Wait: 1 Error: 2 PC: 32613 Cur addr: 7232 Cur len: 720019
too small free_buf: 805
Busy: 1 Wait: 1 Error: 2 PC: 32628 Cur addr: 7232 Cur len: 720019
too small free_buf: 820
Busy: 1 Wait: 1 Error: 2 PC: 32652 Cur addr: 7232 Cur len: 720019
too small free_buf: 844
Busy: 1 Wait: 1 Error: 2 PC: 32678 Cur addr: 7232 Cur len: 720019
too small free_buf: 870
Busy: 1 Wait: 1 Error: 2 PC: 32970 Cur addr: 7232 Cur len: 720019
too small free_buf: 1162
Busy: 1 Wait: 1 Error: 2 PC: 32982 Cur addr: 7232 Cur len: 720019
too small free_buf: 1174
Busy: 1 Wait: 1 Error: 2 PC: 33116 Cur addr: 7232 Cur len: 720019
too small free_buf: 1308
Busy: 1 Wait: 1 Error: 2 PC: 33189 Cur addr: 7232 Cur len: 720019
too small free_buf: 1381
Busy: 1 Wait: 1 Error: 2 PC: 33209 Cur addr: 7232 Cur len: 720019
too small free_buf: 1401
Busy: 1 Wait: 1 Error: 2 PC: 33403 Cur addr: 7232 Cur len: 720019
too small free_buf: 1595
Busy: 1 Wait: 1 Error: 2 PC: 33427 Cur addr: 7232 Cur len: 720019
too 

...

too small free_buf: 1283
Busy: 1 Wait: 1 Error: 2 PC: 45878 Cur addr: 3488 Cur len: 52019
too small free_buf: 1430
Busy: 1 Wait: 1 Error: 2 PC: 46029 Cur addr: 3488 Cur len: 52019
too small free_buf: 1581
Busy: 1 Wait: 1 Error: 2 PC: 46176 Cur addr: 3488 Cur len: 52019
too small free_buf: 1728
Busy: 1 Wait: 1 Error: 2 PC: 46327 Cur addr: 3488 Cur len: 52019
too small free_buf: 1879
Busy: 1 Wait: 1 Error: 2 PC: 46486 Cur addr: 3488 Cur len: 52019
free_buf: 2038
sent 2000 commands to addr 3488, left 50019
Busy: 1 Wait: 1 Error: 2 PC: 47040 Cur addr: 5488 Cur len: 50019
too small free_buf: 592
Busy: 1 Wait: 1 Error: 2 PC: 47070 Cur addr: 5488 Cur len: 50019
too small free_buf: 622
Busy: 1 Wait: 1 Error: 2 PC: 47212 Cur addr: 5488 Cur len: 50019
too small free_buf: 764
Busy: 1 Wait: 1 Error: 2 PC: 47350 Cur addr: 5488 Cur len: 50019
too small free_buf: 902
Busy: 1 Wait: 1 Error: 2 PC: 47506 Cur addr: 5488 Cur len: 50019
too small free_buf: 1058
Busy: 1 Wait: 1 Error: 2 PC: 47651 Cur addr: 

too small free_buf: 1624
Busy: 1 Wait: 1 Error: 2 PC: 62171 Cur addr: 3104 Cur len: 36019
too small free_buf: 1723
Busy: 1 Wait: 1 Error: 2 PC: 62310 Cur addr: 3104 Cur len: 36019
too small free_buf: 1862
Busy: 1 Wait: 1 Error: 2 PC: 62389 Cur addr: 3104 Cur len: 36019
too small free_buf: 1941
Busy: 1 Wait: 1 Error: 2 PC: 62519 Cur addr: 3104 Cur len: 36019
free_buf: 2071
sent 2000 commands to addr 3104, left 34019
Busy: 1 Wait: 1 Error: 2 PC: 63072 Cur addr: 5104 Cur len: 34019
too small free_buf: 624
Busy: 1 Wait: 1 Error: 2 PC: 63152 Cur addr: 5104 Cur len: 34019
too small free_buf: 704
Busy: 1 Wait: 1 Error: 2 PC: 63295 Cur addr: 5104 Cur len: 34019
too small free_buf: 847
Busy: 1 Wait: 1 Error: 2 PC: 63371 Cur addr: 5104 Cur len: 34019
too small free_buf: 923
Busy: 1 Wait: 1 Error: 2 PC: 63511 Cur addr: 5104 Cur len: 34019
too small free_buf: 1063
Busy: 1 Wait: 1 Error: 2 PC: 63656 Cur addr: 5104 Cur len: 34019
too small free_buf: 1208
Busy: 1 Wait: 1 Error: 2 PC: 63792 Cur addr: 

too small free_buf: 1889
Busy: 1 Wait: 1 Error: 2 PC: 12947 Cur addr: 2720 Cur len: 20019
free_buf: 2035
sent 2000 commands to addr 2720, left 18019
Busy: 1 Wait: 1 Error: 2 PC: 13603 Cur addr: 4720 Cur len: 18019
too small free_buf: 691
Busy: 1 Wait: 1 Error: 2 PC: 13776 Cur addr: 4720 Cur len: 18019
too small free_buf: 864
Busy: 1 Wait: 1 Error: 2 PC: 13924 Cur addr: 4720 Cur len: 18019
too small free_buf: 1012
Busy: 1 Wait: 1 Error: 2 PC: 14118 Cur addr: 4720 Cur len: 18019
too small free_buf: 1206
Busy: 1 Wait: 1 Error: 2 PC: 14275 Cur addr: 4720 Cur len: 18019
too small free_buf: 1363
Busy: 1 Wait: 1 Error: 2 PC: 14412 Cur addr: 4720 Cur len: 18019
too small free_buf: 1500
Busy: 1 Wait: 1 Error: 2 PC: 14552 Cur addr: 4720 Cur len: 18019
too small free_buf: 1640
Busy: 1 Wait: 1 Error: 2 PC: 14692 Cur addr: 4720 Cur len: 18019
too small free_buf: 1780
Busy: 1 Wait: 1 Error: 2 PC: 14839 Cur addr: 4720 Cur len: 18019
too small free_buf: 1927
Busy: 1 Wait: 1 Error: 2 PC: 14987 Cur addr

too small free_buf: 1489
Busy: 1 Wait: 1 Error: 2 PC: 26490 Cur addr: 336 Cur len: 6019
too small free_buf: 1578
Busy: 1 Wait: 1 Error: 2 PC: 26618 Cur addr: 336 Cur len: 6019
too small free_buf: 1706
Busy: 1 Wait: 1 Error: 2 PC: 26727 Cur addr: 336 Cur len: 6019
too small free_buf: 1815
Busy: 1 Wait: 1 Error: 2 PC: 26828 Cur addr: 336 Cur len: 6019
too small free_buf: 1916
Busy: 1 Wait: 1 Error: 2 PC: 26965 Cur addr: 336 Cur len: 6019
free_buf: 2053
sent 2000 commands to addr 336, left 4019
Busy: 1 Wait: 1 Error: 2 PC: 27473 Cur addr: 2336 Cur len: 4019
too small free_buf: 561
Busy: 1 Wait: 1 Error: 2 PC: 27575 Cur addr: 2336 Cur len: 4019
too small free_buf: 663
Busy: 1 Wait: 1 Error: 2 PC: 27600 Cur addr: 2336 Cur len: 4019
too small free_buf: 688
Busy: 1 Wait: 1 Error: 2 PC: 27654 Cur addr: 2336 Cur len: 4019
too small free_buf: 742
Busy: 1 Wait: 1 Error: 2 PC: 27715 Cur addr: 2336 Cur len: 4019
too small free_buf: 803
Busy: 1 Wait: 1 Error: 2 PC: 27792 Cur addr: 2336 Cur len: 4019

homing: [<valurap.axes.AxeX1 object at 0xab550fb0>, <valurap.axes.AxeX2 object at 0xaba0e4b0>, <valurap.axes.AxeY object at 0xab6b9790>]
Warning, explicit X in segment
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'X1': -18.45703125, 'Y': 8553.15234375}
homing: [<valurap.axes.AxeX1 object at 0xab550fb0>, <valurap.axes.AxeX2 object at 0xaba0e4b0>, <valurap.axes.AxeY object at 0xab6b9790>]
X1 already on endstop
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'Y': 8553.15234375}
homing: [<valurap.axes.AxeX1 object at 0xab550fb0>, <valurap.axes.AxeX2 object at 0xaba0e4b0>, <valurap.axes.AxeY object at 0xab6b9790>]
X1 already on endstop
X2 already on endstop
Y already on endstop
All axes on endstops, skipping
SetPositions: {'X1': -18.45703125}
Move: {'X1': -3000}
Save position: X1 -19.45703125
homing: [<valurap.axes.AxeX1 object at 0xab550fb0>]
Warning, explicit X in segment
SetPositions: {'X1': -19.45703125}
homing: [<valurap.axes.AxeX1 object at 0

In [ ]:
# check

import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)
imp.reload(emulate)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 1:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True

apg_states = {}
codes = []
first = True
for layer in range(1, 100):
    with open("test5_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    
    print(p[0])
    expected_pos = p[0][1]

    if apg_states:
        print(expected_pos["X"] - apg_states["X"].x / 2**32 / spm)
        print(expected_pos["Y"] - apg_states["Y"].x / 2**32 / spm)

    cms, meta, segments = p[1]
    code = prn.asg.gen_map_code(meta["map"])[:-1]

    pr_opt = []
    
    if first:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], x=int(p[0][1]["X"] * spm)),
                vas.ProfileSegment(apgs["Y"], x=int(p[0][1]["Y"] * spm)),
                vas.ProfileSegment(apgs["Z"], x=0),
            ]]
        ]
        first = False

    
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    if 0:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], v=0),
                vas.ProfileSegment(apgs["Y"], v=0),
                vas.ProfileSegment(apgs["Z"], v=0),
            ]]
        ]

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    code += path_code[:-1]
    print(len(path_code))
    
    emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states, no_tracking=True)
    emu_x = int(apg_states["X"].x) & 0xffffffffff000000
    emu_y = int(apg_states["Y"].x) & 0xffffffffff000000
    
    code += prn.asg.gen_map_code({"Z": "Z"})[:-1]
    code += up_code
    codes.append([expected_pos, (emu_x, emu_y), code])

prn.update_axes_config()

deltas = []
try:
    for exp_pos, (exp_x, exp_y), code in codes:
        prn.exec_long_code(code, splits=2000, verbose=True)
        
        cur_x_hi = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
        cur_y_hi = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
        cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
        cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

        if cur_x_hi < 0:
            cur_x_hi += 2**32

        if cur_y_hi < 0:
            cur_y_hi += 2**32

        if cur_x_lo < 0:
            cur_x_lo += 2**32

        if cur_y_lo < 0:
            cur_y_lo += 2**32
            
        cur_x = cur_x_hi << 32 | cur_x_lo
        cur_y = cur_y_hi << 32 | cur_y_lo

        
        if 0:
            if exp_x > 2 ** 63:
                exp_x -= 2 ** 63

            if exp_y > 2 ** 63:
                exp_y -= 2 ** 63

            if cur_x > 2 ** 63:
                cur_x -= 2 ** 63

            if cur_y > 2 ** 63:
                cur_y -= 2 ** 63

            exp_x /= 2 ** 32 * spm
            exp_y /= 2 ** 32 * spm
            cur_x /= 2 ** 32 * spm
            cur_y /= 2 ** 32 * spm

        print("AAAAAAAA", exp_x, exp_y)
        print("AAAAAAAA", cur_x, cur_y)
        deltas.append([exp_x, cur_x, exp_y, cur_y])

except:
    prn.setup()
    print(deltas)
    raise

    
print(deltas)

In [58]:
for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

SetPositions: {'Z': 5038.6796875}
Move: {'Z': 30000}
Save position: Z 5037.6796875
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
Warning, explicit X in segment
Warning, explicit X in segment
Warning, explicit X in segment
SetPositions: {'X1': 1862.89453125, 'X2': -13503.1484375, 'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
Warning, explicit X in segment
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'X1': 1862.89453125, 'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 0xa87328f0>, <valurap.axes.AxeY object at 0xa87328b0>]
X1 already on endstop
X2 already on endstop
Warning, explicit X in segment
SetPositions: {'Y': -1845.66796875}
homing: [<valurap.axes.AxeX1 object at 0xa8732890>, <valurap.axes.AxeX2 object at 

In [65]:
prn.moveto(X1=-13600)

MoveTo: {'X1': -13600}
Save position: X1 -15000.0546875
Move: {'X1': 1400.0546875}


In [ ]:
from valurap import emulate

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 0:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True
    
pr_opt = []
    
pr_opt += [
    [1, [
        vas.ProfileSegment(apgs["X"], x=0),
        vas.ProfileSegment(apgs["Y"], x=0),
        vas.ProfileSegment(apgs["Z"], x=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], v=-1000000, a=1000000, j=-10000),
        vas.ProfileSegment(apgs["Y"], v=-1000000, a=1000000, j=-10001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], a=-1000000, j=11000),
        vas.ProfileSegment(apgs["Y"], a=-1000000, j=11001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1, [
        vas.ProfileSegment(apgs["X"], v=0),
        vas.ProfileSegment(apgs["Y"], v=0),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
]

acc_step = 10000
code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs=apgs)
prn.exec_long_code(code, splits=2000, verbose=True)
cur_x = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
cur_y = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

if cur_x < 0:
    cur_x += 2**32

if cur_y < 0:
    cur_y += 2**32

if cur_x_lo < 0:
    cur_x_lo += 2**32

if cur_y_lo < 0:
    cur_y_lo += 2**32

apg_states = {}
res = emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states)

print(cur_x << 32 | cur_x_lo)
print(cur_y << 32 | cur_y_lo)
print(int(apg_states["X"].x) & 0xffffffffff000000)
print(int(apg_states["Y"].x) & 0xffffffffff000000)